## Learning Lyapunov function for Inverted Pendulum

In [1]:
# -*- coding: utf-8 -*-
# from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt

c:\Users\dell\anaconda3\envs\tf115\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [2]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=False)
        self.control.weight = torch.nn.Parameter(lqr)

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u

## Dynamical system

In [3]:
def f_value(x,u):
    #Dynamics
    y = []
    G = 9.81  # gravity
    L = 0.5   # length of the pole 
    m = 0.15  # ball mass
    b = 0.1   # friction
    
    for r in range(0,len(x)): 
        f = [ x[r][1], 
              (m*G*L*np.sin(x[r][0])- b*x[r][1]) / (m*L**2)]
        y.append(f) 
    y = torch.tensor(y)
    y[:,1] = y[:,1] + (u[:,0]/(m*L**2))
    return y

## Options

In [4]:
'''
For learning 
'''
N = 500             # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  
x = torch.Tensor(N, D_in).uniform_(-6, 6)           
x_0 = torch.zeros([1, 2])

In [5]:

'''
For verifying 
'''
# x1 = Variable("x1")
# x2 = Variable("x2")
# vars_ = [x1,x2]
G = 9.81 
l = 0.5  
m = 0.15
b = 0.1


In [6]:
# config = Config()
# config.use_polytope_in_forall = True
# config.use_local_optimization = True
# config.precision = 1e-2
# epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.5
ball_ub = 6

## Learning and Falsification

In [33]:
a = 10
b = 12
i = a<b
j = a+3<b
k = i and j
# print(i,j, k)

x = np.array(([1,2],[0,1], [100,200]))
ball = x[:,0] * x[:,0] + x[:,1] * x[:,1]
ball = np.square(x[:,0]) + np.square(x[:,1])
i =  ball < 10
j = ball > 0
print(i, j, np.logical_or(i, j), ball.sum())

[ True  True False] [ True  True  True] [ True  True  True] 50006


In [3]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[-23.58639732,  -5.31421063]])    # lqr solution
    model = Net(D_in,H1, D_out,lqr)
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        V_candidate, u = model(x)
        X0,u0 = model(x_0)
        f = f_value(x,u)
        # print(1, x[:3], 2, u[:3],3, f[:3],4, V_candidate[:3])
        Circle_Tuning = Tune(x)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)
        
        print(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()).size())
        print(L_V.size())
        # With tuning term 
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                    +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()+(X0).pow(2) 
        # Without tuning term
#         Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
        
        
        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()
        q = model.control.weight.data.numpy()

        # Falsification
        # if i % 10 == 0:
        #     u_NN = (q.item(0)*x1 + q.item(1)*x2) 
        #     f = [ x2,
        #          (m*G*l*sin(x1) + u_NN - b*x2) /(m*l**2)]

        #     # Candidate V
        #     z1 = np.dot(vars_,w1.T)+b1

        #     a1 = []
        #     for j in range(0,len(z1)):
        #         a1.append(tanh(z1[j]))
        #     z2 = np.dot(a1,w2.T)+b2
        #     V_learn = tanh(z2.item(0))

        #     print('===========Verifying==========')        
        #     # start_ = timeit.default_timer() 
        #     # result= CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config,epsilon)
        #     # stop_ = timeit.default_timer() 

        #     if (result): 
        #         print("Not a Lyapunov function. Found counterexample: ")
        #         print(result)
        #         x = AddCounterexamples(x,result,10)
        #     else:  
        #         valid = True
        #         print("Satisfy conditions!!")
        #         print(V_learn, " is a Lyapunov function.")
        #     t += (stop_ - start_)
        #     print('==============================') 
        i += 1

    stop = timeit.default_timer()
    
    # torch.save(model.state_dict(), 'Lyapunov_NN.pth')

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

NameError: name 'D_in' is not defined

In [8]:
model = Net(D_in,H1, D_out,lqr)
model.load_state_dict(torch.load('Lyapunov_NN.pth'))
L = []
i = 0
t = 0
max_iters = 2000
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

while i < 2 and not valid: 
    V_candidate, u = model(x)
    X0,u0 = model(x_0)
    f = f_value(x,u)
    print(1, x[:3],'\n', 2, u[:3],'\n',3, f[:3],'\n',4, V_candidate[:3])
    Circle_Tuning = Tune(x)
    # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
    L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                        *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

    # With tuning term 
    Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()\
                +2.2*((Circle_Tuning-6*V_candidate).pow(2)).mean()+(X0).pow(2) 
    # Without tuning term
#         Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
    
    
    print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
    L.append(Lyapunov_risk.item())
    optimizer.zero_grad()
    Lyapunov_risk.backward()
    optimizer.step() 

    w1 = model.layer1.weight.data.numpy()
    w2 = model.layer2.weight.data.numpy()
    b1 = model.layer1.bias.data.numpy()
    b2 = model.layer2.bias.data.numpy()
    q = model.control.weight.data.numpy()
    i+=1


1 tensor([[-0.5030, -0.2057],
        [-2.2500,  1.3803],
        [-3.4326, -1.0581]]) 
 2 tensor([[  8.5099],
        [-11.6274],
        [ 50.6047]], grad_fn=<SliceBackward0>) 
 3 tensor([[-2.0572e-01,  2.1802e+02],
        [ 1.3803e+00, -3.2901e+02],
        [-1.0581e+00,  1.3579e+03]], grad_fn=<SliceBackward0>) 
 4 tensor([[0.1276],
        [0.3477],
        [0.6908]], grad_fn=<SliceBackward0>)
0 Lyapunov Risk= 0.9358062744140625
1 tensor([[-0.5030, -0.2057],
        [-2.2500,  1.3803],
        [-3.4326, -1.0581]]) 
 2 tensor([[  8.5069],
        [-11.6637],
        [ 50.5809]], grad_fn=<SliceBackward0>) 
 3 tensor([[-2.0572e-01,  2.1794e+02],
        [ 1.3803e+00, -3.2998e+02],
        [-1.0581e+00,  1.3573e+03]], grad_fn=<SliceBackward0>) 
 4 tensor([[0.1365],
        [0.3730],
        [0.6610]], grad_fn=<SliceBackward0>)
1 Lyapunov Risk= 0.9809844493865967


### Checking result with smaller epsilon ( Lie derivative of V <= epsilon )

In [19]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)

NameError: name 'vars_' is not defined

### More details on Lyapunov risk
Generally, we start training with Lyapunov risk without the tuning term.      
For example, (1* F.relu(-V_candidate)+ 1.5* F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)    
The weight of each term (1, 1.5, 1.2) can be tuned for balancing each Lyapunov condition.     
Furthermore, using F.relu(L_V+0.5) allows the learning procedure to seek a candidate Lyapunov function with more negative Lie derivative.   
Here 0.5 is also a tunable parameter based on your goal.    
In this example, we use Lyapunov risk with tuning term for achieving large ROA     